In [46]:
from tensorflow.keras.applications import vgg16
import numpy as np
from tensorflow.keras.utils import HDF5Matrix data=HDF5Matrix("data.hdf5","data")


#loading conv layers - aka classifiers
#btw last layer has shape 7x7x512
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

train_dir = './datasets/train'
test_dir = './datasets/test'

nTrain = 200
nTest = 100
batch_size=20

#network output tensor shape
train_features = np.zeros(shape=(nTrain, 7, 7, 512))
train_labels = np.zeros(shape=(nTrain, 3))

test_features = np.zeros(shape=(nTest, 7, 7, 512))
test_labels = np.zeros(shape=(nTest, 3))

datagen = ImageDataGenerator(rescale=1./255) #normalize the data to range from 0 to 1
train_generator = datagen.flow_from_directory( #or hot one encoding for transforming into binary matrix, best modelling of classification problem
    train_dir,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

#iterate through batches
for i, (inputs_batch, labels_batch) in enumerate(train_generator):
    if i*batch_size >= nTrain:
        break
    if i*batch_size < nTest:
        
        features_batch = vgg_conv.predict(inputs_batch)
        train_features[i*batch_size:(i+1)*batch_size] = features_batch
        train_labels[i*batch_size:(i+1)*batch_size] = labels_batch
        
for i, (inputs_batch, labels_batch) in enumerate(test_generator):
    if i*batch_size >= nTest:
        break
    if i*batch_size < nTest:
        features_batch = vgg_conv.predict(inputs_batch)
        test_features[i*batch_size:(i+1)*batch_size] = features_batch
        test_labels[i*batch_size:(i+1)*batch_size] = labels_batch
    
train_features_vec = np.reshape(train_features, (nTrain, 7*7*512))
test_features_vec = np.reshape(test_features, (nTest, 7*7*512))

print("Train features: {}".format(train_features_vec.shape))
print("Test features: {}".format(test_features_vec.shape))

SyntaxError: invalid syntax (<ipython-input-46-bc0026aca656>, line 3)

In [30]:
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import Sequential, optimizers

model = Sequential()
# model.add(vgg_conv)
# model.add(Flatten())
model.add(Dense(512, activation='relu', input_dim=7*7*512))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 512)               12845568  
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 3)                 1539      
Total params: 12,847,107
Trainable params: 12,847,107
Non-trainable params: 0
_________________________________________________________________


In [39]:
### TRAIN
from tensorflow.keras.optimizers import SGD
sgd = SGD(lr=0.01, momentum=0.9, decay=0.01/25, nesterov=False)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features_vec,
                    train_labels,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(test_features_vec, test_labels))

Epoch 1/20
10/10 [==============================] - 1s 73ms/step - loss: 5.8400e-04 - acc: 0.5622 - val_loss: 0.2744 - val_acc: 0.9100
Epoch 2/20
10/10 [==============================] - 1s 73ms/step - loss: 1.2365e-04 - acc: 0.5904 - val_loss: 0.2320 - val_acc: 0.9300
Epoch 3/20
10/10 [==============================] - 1s 82ms/step - loss: 0.0016 - acc: 0.5319 - val_loss: 0.7230 - val_acc: 0.8900
Epoch 4/20
10/10 [==============================] - 1s 87ms/step - loss: 0.0306 - acc: 0.5730 - val_loss: 1.3229 - val_acc: 0.8000
Epoch 5/20
10/10 [==============================] - 1s 87ms/step - loss: 0.3190 - acc: 0.6097 - val_loss: 0.3540 - val_acc: 0.9200
Epoch 6/20
10/10 [==============================] - 1s 84ms/step - loss: 0.4806 - acc: 0.4986 - val_loss: 0.4229 - val_acc: 0.9000
Epoch 7/20
10/10 [==============================] - 1s 84ms/step - loss: 1.0392 - acc: 0.4891 - val_loss: 1.3343 - val_acc: 0.8100
Epoch 8/20
10/10 [==============================] - 1s 90ms/step - loss: 2.

In [ ]:
# CHECK PERFORMANCE

file_names = test_generator.filenames
ground_truth = test_generator.classes
label2index = test_generator.class_indices
idx2label = list(label2index.keys())

print(label2index)
print(f"The list of classes {idx2label}")


In [ ]:
# predicts each img in the array
# predictions = model.predict_classes(test_features_vec)
predictions = np.argmax(model.predict(test_features_vec), axis=-1)
print(predictions)

probs = model.predict(test_features_vec)
print(probs)

err = np.where(predictions != ground_truth)[0]
print(f"Encountered {len(err)} error/s")

In [42]:
scores = model.evaluate(test_features_vec, test_labels, verbose=0)
print("Accuracy: {}".format(scores[1]*100))

Accuracy: 87.00000047683716


In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

for i in range(len(err)):
    pred_class = np.argmax(probs[err[i]])
    pred_label = idx2label[pred_class]
    print("Original label: {}, Prediction: {}, Confidence: {}".format(
        file_names[err[i]].split("/")[0],
        pred_label,
        probs[err[i]][pred_class]
    ))
    original = load_img('{}/{}'.format(test_dir, file_names[err[i]]))
    plt.axis("off")
    plt.imshow(original)
    plt.show()

In [ ]:
def visualize_results(history):
    acc = history.history['acc']
    test_acc = history.history['val_acc']
    loss = history.history['loss']
    test_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'b', label="Training accuracy")
    plt.plot(epochs, test_acc, 'r', label="Test accuracy")
    plt.title('Training and test accuracy')
    
    plt.figure()
    
    plt.plot(epochs, loss, 'b', label="Training loss")
    plt.plot(epochs, test_loss, 'r', label="Test loss")
    plt.title('Training and test loss')
    plt.legend()
    
    plt.show()
    
visualize_results(history)
    
    
    

In [ ]:
# Utility function for obtaining of the errors 
def obtain_errors(val_generator, predictions):
    # Get the filenames from the generator
    fnames = val_generator.filenames

    # Get the ground truth from generator
    ground_truth = val_generator.classes

    # Get the dictionary of classes
    label2index = val_generator.class_indices

    # Obtain the list of the classes
    idx2label = list(label2index.keys())
    print("The list of classes: ", idx2label)

    # Get the class index
    predicted_classes = np.argmax(predictions, axis=1)

    errors = np.where(predicted_classes != ground_truth)[0]
    print("Number of errors = {}/{}".format(len(errors),val_generator.samples))
    
    return idx2label, errors, fnames


# Utility function for visualization of the errors
def show_errors(idx2label, errors, predictions, fnames):
    # Show the errors
    for i in range(len(errors)):
        pred_class = np.argmax(predictions[errors[i]])
        pred_label = idx2label[pred_class]

        title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
            fnames[errors[i]].split('/')[0],
            pred_label,
            predictions[errors[i]][pred_class])

        original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
        plt.figure(figsize=[7,7])
        plt.axis('off')
        plt.title(title)
        plt.imshow(original)
        plt.show()

In [ ]:
predictions = model.predict(test_generator, verbose=1)

idx2label, errors, filenames = obtain_errors(test_generator, predictions)

show_error(idx2label, errors, predictions, filenames)